In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model, load_model
from keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, LSTM, GlobalMaxPooling1D, Dropout 

### **Read Dataset**

In [ ]:
df = pd.read_excel('data_train_intent_detection.xlsx')

In [ ]:
df.head()

,مشکل غیرفعالسازی آوای انتظار,مشکل فعالسازی آوای انتظار,مشکل تغییر آوای انتظار,مشکل پخش آوای انتظار,لیست آواهای انتظار,دریافت کد آوای انتظار خط خود,دریافت کد آوای انتظار خط دیگران,روش فعالسازی آوای انتظار,روش غیرفعالسازی آوای انتظار,روش تغییرآوای انتظار,خارج از دامنه
0,از طریق 8989 امکان لغو اوای انتظار وجود ندارد...,خیلی تلاش کردم اوای انتظار رو فعال کنم اما فع...,سلام علیکم حاج خانوم خوبی من ببخشید مزاحمتون م...,سلام احوال شما خوبه؟ من یه نوحه انتخاب کردم وا...,همه کدای اوای همراه اولو از کجا بگیرم و یکیشو...,یه اهنگ گذاشتم رو خطم نمیدونم کدش چیه واسه دو...,سلام علیکم ما یه نوحه انتخاب کرده بودیم که با ...,الان باهاتون داشتم صحبت میکردم قطع شد من میخوا...,سلام من چطوری میتونم پیشواز خط بابامو بردارم ا...,سلام سال نو مبارک میخواستم ببینم میشه با کدی ک...,سلام من آوای انتظاری که انتخاب کردم تو سایت که...
1,با پیام به 8989 اقدام به غیرفعالسازی اوای انت...,از کجا پیگیری کنم اوای انتظارم فعال نمیشه,سلام حال شما خوب هستین ببخشید من یه پیشوازی بر...,سلام وقتتون بخیر خانومم سه روز پیش یه آوای انت...,همه کدای اوای همراه اولو چجوری بگیرم,یه اهنگ رو خطم گذاشتم میخوام کدشو بهم بدید وا...,سلام من این آهنگ ای دل اگر عاشقی در پی دلدار ب...,سلام من برای فعالسازی خطم وارد سایت همراه من ش...,سلام حال شما خوبه؟من یه چندماهی میشه آوای انتظ...,سلام من خطم بخاطر بدهی قطع شده الان میخوام پیش...,سلام چرا انقدر این پیشواز میزاریم صداش بی‌کیفی...
2,برای غیرفعالسازی اوای انتظار پیام به 8989 داد...,خیلی تلاش کردم اوای انتظار رو فعال کنم اما او...,سلام میکنم خدمتتون خانم عرض ادب غرض از مزاحمت ...,سلام من سه چهار روزه برای خط دخترم پیشواز فعال...,همه کد اواهای انتظار همراه اولو چطور دریافت کنم,دوستم یه اهنگ پیشواز رو خط همراه اولش گذاشته ...,سلام من به رفیقم زنگ زدم بعد نوحه حسین حسین آ...,سلام این کد دستوری همراه اول برای فعالسازی همش...,سلام ببخشید من میخوام آوای انتظار خط همسرمو غی...,سلام خوب هستید؟من میخواستم آوای انتظار گوشی ما...,وقتتون بخیر باشه این آوای انتظاری که من انتخاب...
3,از طریق ارسال پیام به 8989 اقدام به غیرفعالسا...,برای رفع مشکل فعالسازی پیشواز از کجا باید پیگ...,سلام حال شما خوبه این دوست من خیلی به یه خوانن...,سلام خوبید؟من آوای انتظارم اولش میخونه بعدش بو...,کدای پیشواز همشو از کجا بگیرم,پیدا کردن اوام به چه ترتیبیه,سلام من یه چند هفته پیش پیشواز گذاشتم روی خط م...,سلام من چطوری میتونم واسه دوتا از خطهام همزمان...,سلام صبح بخیر،گوشی داداشم خراب شده الان میخواد...,سلام از صبح چند نفر بهم زنگ زدن گفتن چقدر آوای...,سلام من پیشواز از آهنگ طلیسچی گذاشتم صداش خیلی...
4,با ارسال 5 به 8989 میخواستم اوای انتظار غیرفع...,از هر راهی اقدام می کنم اوای انتظار فعال نمیش...,علو خوبین سلام عرض میکنم. عیدتون مبارک. طاعاتت...,سلام من آوای انتظار فعال کردم واسه خطم اما وقت...,همه کدای اوای همراه اول رو از کجا بگیرم و یکی...,رفیق کد انتظارمو ازم خواست یادم نبود چیه,سلام خوبید؟من با همکارم تماس گرفتم بعد آوای ان...,سلام میخواستم بپرسم که برای فعال کردن پیشواز ب...,سلام من تازه پیشواز گذاشتم اما خوشم نیومد میتو...,سلام من آوای انتظارمو از محرم تغییر ندادم همه ...,سلام


In [ ]:
df.shape

(55, 11)

In [ ]:
df.iloc[0]

مشکل غیرفعالسازی آوای انتظار        از طریق 8989 امکان لغو اوای انتظار وجود ندارد...
مشکل فعالسازی آوای انتظار           خیلی تلاش کردم اوای انتظار رو فعال کنم اما فع...
مشکل تغییر آوای انتظار             سلام علیکم حاج خانوم خوبی من ببخشید مزاحمتون م...
مشکل پخش آوای انتظار               سلام احوال شما خوبه؟ من یه نوحه انتخاب کردم وا...
لیست آواهای انتظار                  همه کدای اوای همراه اولو از کجا بگیرم و یکیشو...
دریافت کد آوای انتظار خط خود        یه اهنگ گذاشتم رو خطم نمیدونم کدش چیه واسه دو...
دریافت کد آوای انتظار خط دیگران    سلام علیکم ما یه نوحه انتخاب کرده بودیم که با ...
روش فعالسازی آوای انتظار           الان باهاتون داشتم صحبت میکردم قطع شد من میخوا...
روش غیرفعالسازی آوای انتظار        سلام من چطوری میتونم پیشواز خط بابامو بردارم ا...
روش تغییرآوای انتظار               سلام سال نو مبارک میخواستم ببینم میشه با کدی ک...
خارج از دامنه                      سلام من آوای انتظاری که انتخاب کردم تو سایت که...
Name: 0, dtype: object

In [ ]:
list(df.iloc[0].keys())

['مشکل غیرفعالسازی آوای انتظار',
 'مشکل فعالسازی آوای انتظار',
 'مشکل تغییر آوای انتظار',
 'مشکل پخش آوای انتظار',
 'لیست آواهای انتظار',
 'دریافت کد آوای انتظار خط خود',
 'دریافت کد آوای انتظار خط دیگران',
 'روش فعالسازی آوای انتظار',
 'روش غیرفعالسازی آوای انتظار',
 'روش تغییرآوای انتظار',
 'خارج از دامنه']

In [ ]:
def create_dataset(df):
  output = {
        "text": [],
        "category": []
    }

  for data in df.iloc:
    for i,d in enumerate(data):
      output["text"].append(d)
      output["category"].append(list(df.iloc[0].keys())[i])
   
  return output

In [ ]:
data = create_dataset(df)

In [ ]:
print(len(data["text"]))
print(len(data["category"]))

605
605


In [ ]:
print(data["text"][0])
print(data["category"][0])

 از طریق 8989 امکان لغو اوای انتظار وجود ندارد از چه روشی می توانم پیگیری کنم
مشکل غیرفعالسازی آوای انتظار


In [ ]:
dataset = pd.DataFrame(data)

In [ ]:
dataset.head()

,text,category
0,از طریق 8989 امکان لغو اوای انتظار وجود ندارد...,مشکل غیرفعالسازی آوای انتظار
1,خیلی تلاش کردم اوای انتظار رو فعال کنم اما فع...,مشکل فعالسازی آوای انتظار
2,سلام علیکم حاج خانوم خوبی من ببخشید مزاحمتون م...,مشکل تغییر آوای انتظار
3,سلام احوال شما خوبه؟ من یه نوحه انتخاب کردم وا...,مشکل پخش آوای انتظار
4,همه کدای اوای همراه اولو از کجا بگیرم و یکیشو...,لیست آواهای انتظار


In [ ]:
dataset.shape

(605, 2)

In [ ]:
dataset['category'].value_counts()

مشکل غیرفعالسازی آوای انتظار       55
مشکل فعالسازی آوای انتظار          55
مشکل تغییر آوای انتظار             55
مشکل پخش آوای انتظار               55
لیست آواهای انتظار                 55
دریافت کد آوای انتظار خط خود       55
دریافت کد آوای انتظار خط دیگران    55
روش فعالسازی آوای انتظار           55
روش غیرفعالسازی آوای انتظار        55
روش تغییرآوای انتظار               55
خارج از دامنه                      55
Name: category, dtype: int64

In [ ]:
#delete NaN
dataset = dataset.dropna()

In [ ]:
dataset.shape

(498, 2)

**stopword**

In [ ]:
def read_stop_words(filename):
  with open(filename) as stopwords_file:
    stopwords = stopwords_file.readlines()
  stopwords = [line.replace('\n', '') for line in stopwords] 
  return stopwords

In [ ]:
stopwords = read_stop_words('stopwords.txt')
print(len(stopwords))

1316


In [ ]:
# install hazm library
!pip install hazm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.7/316.7 KB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 KB 22.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 74.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394488 sha256=4b99bc04b729360eb3c8e9a447e07ea0475b85ef7e38a43a486fdf1ef8dbaada
  Stored in directory: /root/.cache/pip/wheels/ac/62/f6/88933dadcd64a1614894614aa68cf57c4b8e5256acb650b1f1
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp39-cp39-linux_x86_64.whl size=180371 sha256=13dda83bc6266db6d8a51af098a31ccc143c9c8c79c67ef467d9926d75e0cbba
  Stored in directory: /root/.cache/pip/wheels/8e/ff/82/9326b96f96f47472e02c453697b225813e4650c0ed4df2cd49
Successfully built nltk libwapiti
  Attempting unin

In [ ]:
from hazm import word_tokenize

In [ ]:
text = dataset['text'].values

In [ ]:
text[0]

' از طریق 8989 امکان لغو اوای انتظار وجود ندارد از چه روشی می توانم پیگیری کنم'

In [ ]:
text_preprocessing = [re.sub('/s', '', line) for line in text]

In [ ]:
word_tokenized = [word_tokenize(t) for t in text]
word_tokenized_filtered = [[w for w in sentence if w not in stopwords] for sentence in word_tokenized]

In [ ]:
print(word_tokenized[0])

['از', 'طریق', '8989', 'امکان', 'لغو', 'اوای', 'انتظار', 'وجود', 'ندارد', 'از', 'چه', 'روشی', 'می', 'توانم', 'پیگیری', 'کنم']


In [ ]:
print(word_tokenized_filtered[0])

['8989', 'امکان', 'لغو', 'اوای', 'انتظار', 'روشی', 'پیگیری']


In [ ]:
def concat_words_of_sentence(word_tokenized_filtered):
  corpus = []
  for sentence in word_tokenized_filtered:
    s = ' '
    for w in sentence:
      s += w + ' '
    corpus.append(s)  
  return corpus

In [ ]:
corpus = concat_words_of_sentence(word_tokenized_filtered)

In [ ]:
print(corpus[3])

 سلام احوال خوبه نوحه انتخاب واسه پیشوازم پخش نمیشه خطم بوق ساده میخوره چیکار 


### **one hot Y**

In [ ]:
Y_one_hot = pd.get_dummies(dataset['category'])

In [ ]:
category_types = Y_one_hot.columns.values

In [ ]:
category_types

array(['خارج از دامنه', 'دریافت کد آوای انتظار خط خود',
       'دریافت کد آوای انتظار خط دیگران', 'روش تغییرآوای انتظار',
       'روش غیرفعالسازی آوای انتظار', 'روش فعالسازی آوای انتظار',
       'لیست آواهای انتظار', 'مشکل تغییر آوای انتظار',
       'مشکل غیرفعالسازی آوای انتظار', 'مشکل فعالسازی آوای انتظار',
       'مشکل پخش آوای انتظار'], dtype=object)

In [ ]:
Y = pd.get_dummies(dataset['category']).values

In [ ]:
Y[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], dtype=uint8)

### **Train & Test Split**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(corpus, Y, shuffle=True, test_size=0.3, random_state=42)

In [ ]:
print(np.array(X_train).shape)
print(np.array(X_test).shape)
print(np.array(y_train).shape)
print(np.array(y_test).shape)

(348,)
(150,)
(348, 11)
(150, 11)


### **Train The Model**

In [ ]:
!pip install transformers
import transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 82.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 77.8 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer,TFBertModel
MODEL_NAME = "HooshvareLab/bert-fa-zwnj-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
bert = TFBertModel.from_pretrained(MODEL_NAME)

Some layers from the model checkpoint at HooshvareLab/bert-fa-zwnj-base were not used when initializing TFBertModel: ['mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFBertModel were not initialized from the model checkpoint at HooshvareLab/bert-fa-zwnj-base and are newly initialized: ['bert/pooler/dense/kernel:0', 'bert/pooler/dense/bias:0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
bert.summary()

Model: "tf_bert_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  118297344 
                                                                 
Total params: 118,297,344
Trainable params: 118,297,344
Non-trainable params: 0
_________________________________________________________________


In [ ]:
x_train = tokenizer(
    text=X_train,
    add_special_tokens=True,
    max_length=70,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

x_test = tokenizer(
    text=X_test,
    add_special_tokens=True,
    max_length=70,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

In [ ]:
input_ids = x_train['input_ids']
attention_mask = x_train['attention_mask']

In [ ]:
input_ids[0]

<tf.Tensor: shape=(41,), dtype=int32, numpy=
array([    2, 10248, 14980,  4508,  2446,  1113, 11131, 13605, 22680,
       21093,     3,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0], dtype=int32)>

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense

In [ ]:
max_len = 41
input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
input_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")
embeddings = bert(input_ids,attention_mask = input_mask)[0] 
out = tf.keras.layers.GlobalMaxPool1D()(embeddings)
out = Dense(128, activation='relu')(out)
out = tf.keras.layers.Dropout(0.1)(out)
out = Dense(32, activation = 'relu')(out)
y = Dense(len(category_types), activation = 'sigmoid')(out)
model = tf.keras.Model(inputs=[input_ids, input_mask], outputs=y)
model.layers[2].trainable = True

In [ ]:
# optimizer = Adam(
#     learning_rate=5e-05, # this learning rate is for bert model , taken from huggingface website 
#     epsilon=1e-08,
#     decay=0.01,
#     clipnorm=1.0)

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 41)]         0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 41)]         0           []                               
                                                                                                  
 tf_bert_model_1 (TFBertModel)  multiple             118297344   ['input_ids[0][0]',              
                                                                  'attention_mask[0][0]']         
                                                                                                  
 global_max_pooling1d_4 (Global  (None, 768)         0           ['tf_bert_model_1[1][0]']  

In [ ]:
train_history = model.fit(
    x ={'input_ids':x_train['input_ids'],'attention_mask':x_train['attention_mask']} ,
    y = y_train,
    validation_data = (
    {'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']}, y_test
    ),
    epochs=20,
    batch_size=36
)

Epoch 1/20


10/10 [==============================] - 55s 879ms/step - loss: 3.5665 - accuracy: 0.0862 - val_loss: 2.5935 - val_accuracy: 0.0867
Epoch 2/20
10/10 [==============================] - 5s 446ms/step - loss: 2.4464 - accuracy: 0.1149 - val_loss: 2.4365 - val_accuracy: 0.0867
Epoch 3/20
10/10 [==============================] - 4s 394ms/step - loss: 2.4010 - accuracy: 0.1063 - val_loss: 2.4174 - val_accuracy: 0.0867
Epoch 4/20
10/10 [==============================] - 5s 468ms/step - loss: 2.4289 - accuracy: 0.1092 - val_loss: 2.4431 - val_accuracy: 0.0867
Epoch 5/20
10/10 [==============================] - 4s 450ms/step - loss: 2.3980 - accuracy: 0.1264 - val_loss: 2.4495 - val_accuracy: 0.0867
Epoch 6/20
10/10 [==============================] - 4s 390ms/step - loss: 2.4115 - accuracy: 0.1092 - val_loss: 2.3979 - val_accuracy: 0.0867
Epoch 7/20
10/10 [==============================] - 4s 421ms/step - loss: 2.4003 - accuracy: 0.0948 - val_loss: 2.3976 - val_accuracy: 0.1000
Epoch 8/20
10/10

In [ ]:
model.save('model.h5')

In [ ]:
predicted_raw = model.predict({'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']})
predicted_raw[0]

5/5 [==============================] - 4s 98ms/step


array([0.45642886, 0.45023513, 0.47857657, 0.48195222, 0.5263352 ,
       0.53656805, 0.5672045 , 0.5666342 , 0.39901245, 0.42741755,
       0.45654598], dtype=float32)